In [5]:
#Task 1: Data Quality Assessment (20 Points)

import pandas as pd

# Load the data
df = pd.read_csv("ecommerce_customers_large.csv")

# Peek at the top 5 rows
print(df.head())

# Basic info
print(df.info())
print("Shape:", df.shape)
print("Memory Usage:")
print(df.memory_usage(deep=True))
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_report = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage (%)': missing_percent
})
print(missing_report)
print(df.dtypes)
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count}")
print(df.describe())

   CustomerID  Age  Gender  AnnualIncome  SpendingScore BrowserType  \
0        1001   23    Male        118266             98        Edge   
1        1002   57  Female         71930             39        Edge   
2        1003   50  Female        113230             43       Opera   
3        1004   50  Female         69083             15      Chrome   
4        1005   50  Female        109557             52     Firefox   

   PurchaseFrequency  LastPurchaseDaysAgo DeviceType  
0                  4                   31     Tablet  
1                 12                   10     Mobile  
2                 16                    9     Mobile  
3                  1                   50     Mobile  
4                 13                    4    Desktop  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   CustomerID           120 non-null   

In [6]:
#Task 2: Data Cleaning (40 Points)

# Analyze missing values
missing = df.isnull().sum()
print("Missing value count:\n", missing)

# Strategy: example – fill numeric with median, categorical with mode
df['SpendingScore'].fillna(df['SpendingScore'].median(), inplace=True)
df['BrowserType'].fillna(df['BrowserType'].mode()[0], inplace=True)
# Exact duplicates
duplicates = df[df.duplicated()]
print(f"Exact duplicates: {len(duplicates)}")

# Drop exact duplicates
df.drop_duplicates(inplace=True)

# For near-duplicates or merge conflicts, use heuristics (e.g., recent purchases)
# Example placeholder:
# df.sort_values(by='LastPurchaseDaysAgo', inplace=True)
# Check data types
print(df.dtypes)

# Fix common issues
df['AnnualIncome'] = pd.to_numeric(df['AnnualIncome'], errors='coerce')
df['PurchaseFrequency'] = pd.to_numeric(df['PurchaseFrequency'], errors='coerce')

# Standardize categories (e.g., browser/device names)
df['BrowserType'] = df['BrowserType'].str.strip().str.title()
df['DeviceType'] = df['DeviceType'].str.strip().str.title()
# Capitalize standardized names
df['Gender'] = df['Gender'].str.capitalize()

# Remove whitespace from text fields
df['BrowserType'] = df['BrowserType'].str.strip()
df['DeviceType'] = df['DeviceType'].str.strip()

Missing value count:
 CustomerID             0
Age                    0
Gender                 0
AnnualIncome           0
SpendingScore          0
BrowserType            0
PurchaseFrequency      0
LastPurchaseDaysAgo    0
DeviceType             0
dtype: int64
Exact duplicates: 0
CustomerID              int64
Age                     int64
Gender                 object
AnnualIncome            int64
SpendingScore           int64
BrowserType            object
PurchaseFrequency       int64
LastPurchaseDaysAgo     int64
DeviceType             object
dtype: object


C:\Users\HomePC\AppData\Local\Temp\ipykernel_20768\2434655338.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SpendingScore'].fillna(df['SpendingScore'].median(), inplace=True)
C:\Users\HomePC\AppData\Local\Temp\ipykernel_20768\2434655338.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [24]:
#Task 3: Outlier Detection and Treatment (20 Points)

import pandas as pd
import numpy as np
from scipy import stats

# Load cleaned data
df = pd.read_csv("ecommerce_customers_large.csv")

# Select only numeric columns
num_cols = df.select_dtypes(include=np.number).columns

# Store results for documentation
outlier_summary = {}

for col in num_cols:
    col_data = df[col].dropna()
    
    # IQR Method
    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    iqr_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    
    # Z-Score Method
    z_scores = np.abs(stats.zscore(col_data))
    zscore_outliers = df[(z_scores > 3)][col]  # Z-score threshold = 3
    
    # Combine
    total_outliers = set(iqr_outliers.index).union(set(zscore_outliers.index))
    outlier_count = len(total_outliers)
    total_count = df.shape[0]
    percent = round((outlier_count / total_count) * 100, 2)

    # Decision logic (example): Cap if <5%, else flag for review
    if percent < 5:
        df[col] = np.where(df[col] > upper_bound, upper_bound,
                           np.where(df[col] < lower_bound, lower_bound, df[col]))
        decision = "Capped"
    else:
        decision = "Needs Review"

    # Document decisions
    outlier_summary[col] = {
        "Outlier_Count": outlier_count,
        "Total_Rows": total_count,
        "Percent": percent,
        "Decision": decision,
        "IQR_Bounds": (round(lower_bound, 2), round(upper_bound, 2))
    }

# Save treated dataset
df.to_csv("outlier_treated_ecommerce_customers.csv", index=False)

# Print documentation
print("📊 Outlier Treatment Summary:")
for col, info in outlier_summary.items():
    print(f"\n▶ Column: {col}")
    for key, val in info.items():
        print(f"   - {key}: {val}")

📊 Outlier Treatment Summary:

▶ Column: CustomerID
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(941.5), np.float64(1179.5))

▶ Column: Age
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(1.0), np.float64(83.0))

▶ Column: AnnualIncome
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(-29345.62), np.float64(165217.38))

▶ Column: SpendingScore
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(-30.5), np.float64(125.5))

▶ Column: PurchaseFrequency
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(-9.0), np.float64(31.0))

▶ Column: LastPurchaseDaysAgo
   - Outlier_Count: 0
   - Total_Rows: 120
   - Percent: 0.0
   - Decision: Capped
   - IQR_Bounds: (np.float64(-39.5), np.flo

In [21]:
# Task 4:
# ---------------------------------
# Begin Feature Engineering
# ---------------------------------
feature_data = df.copy()

# 1. Customer Tenure (simulate registration date)
np.random.seed(42)
feature_data["RegistrationDate"] = pd.to_datetime(
    np.random.choice(pd.date_range("2015-01-01", "2022-01-01"), size=len(feature_data))
)
feature_data["CustomerTenureDays"] = (pd.Timestamp.today() - feature_data["RegistrationDate"]).dt.days

# 2. Average Order Value
feature_data["AvgOrderValue"] = feature_data["AnnualIncome"] / feature_data["PurchaseFrequency"]
feature_data["AvgOrderValue"].replace([np.inf, -np.inf], np.nan, inplace=True)

# 3. Days Since Last Order (renamed)
feature_data.rename(columns={"LastPurchaseDaysAgo": "DaysSinceLastOrder"}, inplace=True)

# 4. Age Groups
feature_data["AgeGroup"] = pd.cut(
    feature_data["Age"],
    bins=[0, 18, 30, 45, 60, 100],
    labels=["<18", "18-30", "31-45", "46-60", "60+"]
)

# 5. Income Brackets (quartiles)
feature_data["IncomeBracket"] = pd.qcut(
    feature_data["AnnualIncome"],
    q=4,
    labels=["Low", "Mid", "High", "Very High"]
)

# 6. Spending Categories (based on SpendingScore)
feature_data["SpendingCategory"] = pd.cut(
    feature_data["SpendingScore"],
    bins=[-1, 30, 60, 100],
    labels=["Low", "Medium", "High"]
)

# 7. Interaction Feature: Income Per Order
feature_data["IncomePerOrder"] = feature_data["AnnualIncome"] / feature_data["PurchaseFrequency"]
feature_data["IncomePerOrder"].replace([np.inf, -np.inf], np.nan, inplace=True)

# ---------------------------------
# View Result
# ---------------------------------
print(feature_data[[
    "CustomerID", "CustomerTenureDays", "AvgOrderValue", "DaysSinceLastOrder",
    "Age", "AgeGroup", "AnnualIncome", "IncomeBracket",
    "SpendingScore", "SpendingCategory", "IncomePerOrder"
]])

     CustomerID  CustomerTenureDays  AvgOrderValue  DaysSinceLastOrder  Age  \
0          1001                2995   29545.410000                  31   23   
1          1002                2561    5994.166667                  10   57   
2          1003                2725    7076.875000                   9   50   
3          1004                2760   69083.000000                  50   50   
4          1005                2217    8427.461538                   4   50   
..          ...                 ...            ...                 ...  ...   
115        1116                3649   10645.833333                   2   43   
116        1117                2817    8126.666667                   6   58   
117        1118                1974    4234.055556                  38   59   
118        1119                2371    4394.714286                   5   48   
119        1120                2992    8622.125000                  47   56   

    AgeGroup  AnnualIncome IncomeBracket  SpendingS

C:\Users\HomePC\AppData\Local\Temp\ipykernel_20768\3760135994.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feature_data["AvgOrderValue"].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\HomePC\AppData\Local\Temp\ipykernel_20768\3760135994.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 